## ⚙️ 1. Setup: Install Libraries

First, let's install the specific version of the Google Agent Development Kit (ADK) that this notebook is built with. Pinning the version ensures our code will always work as expected.

In [1]:
!pip install google-adk==1.13.0 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 5.1 MB/s eta 0:00:00


## 🔑 2. Authentication: Configure Your API Key

Next, we need to securely provide our Google API key. This code will create a secure input prompt for you to paste your key. It then sets the key as an environment variable, which is the standard way the ADK authenticates your requests.

In [16]:
import os
from getpass import getpass

# Prompt the user for their API key securely
api_key = getpass('Enter your Google API Key: ')

# Set the API key as an environment variable for ADK to use
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully! Let the fun begin.")

Enter your Google API Key: ··········
✅ API Key configured successfully! Let the fun begin.


## 🛠️ 3. Define Workflow Tools

For our iterative workflow, we need two key functions: sum_costs for our accountant agent to do math, and exit_loop for our refiner agent to signal when the goal is met and the loop should terminate.

In [24]:
import json
from google.adk.tools import ToolContext

#A tool to sum costs
def sum_costs(costs: list[float]) -> float:
  """Calculates the sum of a list of numbers."""
  print(f"  [Tool Call] sum_costs on the list: {costs}")
  return sum(costs)

# A tool to signal that the loop should terminate.
def exit_loop(tool_context: ToolContext):
  """Call this function ONLY when the plan is approved and within budget."""
  print(f"  [Tool Call] Budget approved. Terminating loop: {json.dumps(tool_context.state.to_dict())}")
  tool_context.actions.escalate = True
  return {"result" : "Budget approved. Finalizing plan."}

## 4. Create Tool Wrappers

To ensure our cost_cutter_agent can reliably use both the built-in Google Search and our custom exit_loop function, we'll wrap google search agent as a tool agent

In [25]:
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.tools.agent_tool import AgentTool

google_search_agent = Agent(
    name="Google_Search_agent",
    model="gemini-2.5-flash",
    instruction="You are just a wrapper for the Google Search tool.",
    tools=[google_search]
)

google_search_tool = AgentTool(agent=google_search_agent)

## 📝 5. Create the Proposer, Accountant, Cost Cutter and Plan Retriever Agents

###Proposer:
This is the first agent in our workflow. It runs only once to propose an initial plan based on the user's topic. Its output is stored in the current_plan variable.

###Accountant
This agent that runs at the beginning of each loop. It takes the current_plan, checks it against the budget using its sum_costs tool, and outputs a critique.

###Cost Cutter
This agent is the second step in our loop. It takes the critique as input. If the budget is not met, it uses its Google Search Tool to find a cheaper option. If the budget is met, it calls the exit_loop tool to terminate the process.

###Plan Retriever
This agent's only job is to run after the loop has successfully finished. It takes the final current_plan from the session state and presents it clearly to the user.

In [27]:
global COMPLETION_PHRASE
COMPLETION_PHRASE = "The plan is within the budget."

# Agent 1: Proposes the initial, expensive plan (runs once).
spending_proposer_agent = Agent(
    name="spending_proposer_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="""
    You are a luxury event planner. For a {{topic}}, find a high-end venue and a gourmet catering service.

    Output a JSON object with items and their estimated costs, like:
    {"venue": {"name": "The Ritz London", "cost": 10000}, "catering": {"name": "Gourmet Chefs Inc.", "cost": 5000}}
    """,
    output_key="current_plan",
    #after_agent_callback = log_output_after_agent
)

# Agent 2 (in loop): The "Accountant" that critiques the plan.
# COMPLETION_PHRASE = "The plan is within the budget."

accountant_agent = Agent(
    name="accountant_agent",
    model="gemini-2.5-flash",
    tools=[sum_costs],
    instruction="""
    You are a meticulous accountant. Your budget is {{budget}}.
    The current plan is: {{current_plan}}

    Extract the costs from the plan and use the `sum_costs` tool to get the total.
    - IF the total cost is > {{budget}}, output a critique like: "This plan is over budget by [amount]. Find a cheaper [item]."
    - ELSE, respond with the exact phrase: '{COMPLETION_PHRASE}'
    """,
    output_key="critique"
)

# Agent 3 (in loop): The "Cost Cutter" that refines the plan.
cost_cutter_agent = Agent(
    name="cost_cutter_agent",
    model="gemini-2.5-flash",
    tools=[google_search_tool, exit_loop],
    instruction="""
    You are a cost-cutting expert. You must refine a plan based on a critique.
    The critique is: {{critique}}
    The current plan is: {{current_plan}}

    - IF the critique is '{COMPLETION_PHRASE}', you MUST call the `exit_loop` tool with no arguments.
    - ELSE, read the critique to identify the overpriced item. Use your search tool to find a cheaper alternative for that item.
      Output a new JSON object with the updated plan.
    """,
    output_key="current_plan"  # This overwrites the plan for the next loop iteration.
)

#Agent 4: Runs once after the loop finishes to present the final plan.
plan_retriever_agent = Agent(
    name="plan_retriever_agent",
    model="gemini-2.5-flash",
    instruction="""
    You are a plan finalizer. Your only job is to present the final, approved plan.
    The plan is available in the context variable `{{current_plan}}`.

    Your output must be the content of the final plan presented in a clear and easy-to-read format".
    """,
    tools=[]
)

## 🔄 6. Assemble the Loop and Sequential Workflows

Now we assemble our agent team. The LoopAgent is created to handle the critique-and-refine cycle. Then, a top-level SequentialAgent puts the entire "propose -> loop -> present" workflow together.

In [28]:
from google.adk.agents import LoopAgent, SequentialAgent

budget_refinement_loop = LoopAgent(
    name="budget_refinement_loop",
    sub_agents=[accountant_agent, cost_cutter_agent],
    max_iterations=3
    )


budget_optimizer_workflow = SequentialAgent(
    name="budget_optimizer_workflow",
    sub_agents=[spending_proposer_agent, budget_refinement_loop, plan_retriever_agent]
)

## 🚀 7. Build the Execution Engine

This is our helper function for running queries, unchanged from previous articles. It handles the core ADK logic of initializing the Runner and streaming events.

In [32]:
from IPython.display import display, Markdown

from google.adk.sessions import Session
from google.genai.types import Content, Part
from google.adk.runners import Runner

async def run_agent_query(agent: Agent, query: str, topic: str, budget: str, session: Session, user_id: str):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user"),
            state_delta={'budget': budget, 'topic': topic, 'COMPLETION_PHRASE': COMPLETION_PHRASE} # Pass budget and topic to context
        ):
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    print("\n" + "-"*50)
    print("✅ Final Response:")
    display(Markdown(final_response))
    print("-"*50 + "\n")

    return final_response


## ✨ 8. Initialize Session and Run the Workflow

Finally, we set up our InMemorySessionService and define our main execution block. We set the initial budget and topic in the session state, then kick off the entire autonomous workflow with a single call.

In [33]:
from google.adk.sessions import InMemorySessionService


# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.
session_service = InMemorySessionService()
user_id = "adk_event_planner_001"

In [34]:
async def run_stateful_orchestrator():

  session = await session_service.create_session(
        app_name=budget_optimizer_workflow.name,
        user_id=user_id
  )

  budget=15000
  topic="50 person AI event in New York"
  query = f"Find a plan for {topic}"

  print(f"User: {query}\n")
  await run_agent_query(budget_optimizer_workflow, query, topic, budget, session, user_id)

# Run the full system
await run_stateful_orchestrator()

User: Find a plan for 50 person AI event in New York


🚀 Running query for agent: 'budget_optimizer_workflow' in session: 'c093d651-5626-49a1-b255-86be79d3b6f0'...


  [Tool Call] sum_costs on the list: [30000, 12500]
  [Tool Call] sum_costs on the list: [2500, 12500]
  [Tool Call] Budget approved. Terminating loop: {"budget": 15000, "topic": "50 person AI event in New York", "COMPLETION_PHRASE": "The plan is within the budget.", "current_plan": "```json\n{\n  \"venue\": {\n    \"name\": \"OASIS by Workville\",\n    \"cost\": 2500\n  },\n  \"catering\": {\n    \"name\": \"Sweet Basil Catering\",\n    \"cost\": 12500\n  }\n}\n```", "critique": "The plan is within the budget."}

--------------------------------------------------
✅ Final Response:


The final approved plan for your 50-person AI event in New York is as follows:

### Event Plan: 50-Person AI Event in New York

**Venue:**

*   **Name:** OASIS by Workville
*   **Description:** This venue offers private full-floor meeting spaces, ideal for teams of 10-50 guests. It features open desk workstations, conference areas with 65-inch display monitors, and high-speed Wi-Fi, making it perfectly suited for an AI event. A rooftop option is also available.
*   **Estimated Cost:** $2,500

**Catering Service:**

*   **Name:** Sweet Basil Catering
*   **Description:** A full-service catering company with 17 years of experience, known for custom, seasonal, and exquisite menus. They provide a complete end-to-end catering experience, including food, beverages, professional staffing, rental coordination, setup, and cleanup, with an ability to tailor menus to dietary preferences.
*   **Estimated Cost:** $12,500

**Total Estimated Cost:** $15,000

This plan is within the allocated budget.

--------------------------------------------------

